<a href="https://colab.research.google.com/github/NoeGille/UNet-on-fashion-mnist/blob/main/Neural_network_with_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports



In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

## Create fully connected network

In [35]:
class NN(nn.Module):

    # Constructor

    def __init__(self, input_size, num_classes): 
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x



## Set device

In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Hyperparameters

In [37]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
epochs = 5

## Load Data

In [38]:
train_dataset = datasets.MNIST(root='tmp/dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='tmp/dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


## Initialize networks

In [39]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

## Loss and optimizer

In [40]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Train network

In [44]:
from tqdm import tqdm

for epoch in range(epochs):
  for (data, targets) in tqdm(train_loader):
    # get data on cuda if possible
    data = data.to(device=device)
    targets = targets.to(device=device)
    # data.shape = (64, 1, 28, 28)
    # we want (64, 28, 28, 1)
    data = data.reshape(data.shape[0],-1)

    # forward 
    # prediction
    scores = model(data)
    # Calculate loss 
    loss = criterion(scores, targets)

    # backward
    optimizer.zero_grad()
    loss.backward()

    #gradient descent or adam step
    optimizer.step()



100%|██████████| 938/938 [00:09<00:00, 103.22it/s]


## Check accuracy on training and test to see how good our model

In [45]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            # Getting the index of the maximum value
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        acc = round(float(num_correct) / float(num_samples)*100, ndigits=2)
        print(f"Got {num_correct} / {num_samples} with accuracy {acc}")
      
    model.train()
    return acc

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 59013 / 60000 with accuracy 98.36
Checking accuracy on test data
Got 9713 / 10000 with accuracy 97.13


97.13